In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import numpy as np

df_demograficos = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIDemograficosTt_DATA_LABELS_2023-01-24_1922 - BDIPMamaV11-INTELIDemograficosTt_DATA_LABELS_2023-01-24_1922genesis.csv")
df_histopatologia = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIHistopatologia_DATA_LABELS_2023-01-24_1924.csv")
df_registro_tumo = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIRegistroDeTumo_DATA_LABELS_2023-01-24_1924.csv")
df_pesoEaltura = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.csv")

In [16]:
df_demograficos = df_demograficos.drop(['Repeat Instrument', 
                                        'Repeat Instance',
                                        'UF de nascimento do paciente',
                                        'UF de residência do paciente',
                                        'Quantas vezes ficou gráfida?',
                                        'Número de partos',
                                        'Hormonioterapia ',
                                        'Amamentou na primeira gestação?',
                                        'Faz uso de métodos contraceptivo?',
                                        'Esquema de hormonioterapia',
                                        'Esquema de hormonioterapia',
                                        'Data do início Hormonioterapia adjuvante',
                                        'Data de início da Radioterapia',
                                        'Radioterapia',
                                        'Consumo de tabaco',
                                        'Abortou'], axis=1)

In [17]:
df_demograficos = df_demograficos.rename(columns={
                        "Idade do paciente ao primeiro diagnóstico": "idade_diagnostico", 
                        "Raça declarada (Biobanco)": "raca",
                        "Última informação do paciente": "ultima_informacao_paciente",
                        "Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]": "tempo_seguimento",
                        "Já ficou grávida?": "ja_gravida",
                        "Idade na primeira gestação": "gestacao_idade",
                        "Consumo de álcool": "consumo_alcool",
                        "Possui histórico familiar de câncer?": "historico_cancer",
                        "Grau de parentesco de familiar com cancer? (choice=primeiro (pais, irmãos, filhos))": "grau_parentesco_primeiro",
                        "Grau de parentesco de familiar com cancer? (choice=segundo (avós, tios e netos))": "grau_parentesco_segundo",
                        "Grau de parentesco de familiar com cancer? (choice=terceiro (bisavós, tio avós, primos, sobrinhos))": "grau_parentesco_terceiro",
                        "Regime de Tratamento": "tratamento",
                        "Tipo de terapia anti-HER2 neoadjuvante": "anti_her2_neoadjuvante"
                        })

df_demograficos['Escolaridade'].replace( 'IGNORADA', 0, inplace=True )
df_demograficos['Escolaridade'].replace( 'ANALFABETO', 1, inplace=True )
df_demograficos['Escolaridade'].replace( 'ENS. FUNDAMENTAL INCOMPLETO', 2, inplace=True )
df_demograficos['Escolaridade'].replace( 'ENS. FUNDAMENTAL COMPLETO', 3, inplace=True )
df_demograficos['Escolaridade'].replace( 'ENSINO MÉDIO', 4, inplace=True )
df_demograficos['Escolaridade'].replace( 'SUPERIOR', 5, inplace=True )

df_demograficos['Sexo'].replace( 'Feminino', 0, inplace=True )
df_demograficos['Sexo'].replace( 'Masculino', 1, inplace=True )

df_demograficos['raca'].replace( 'Negro', 0, inplace=True )
df_demograficos['raca'].replace( 'Pardo', 1, inplace=True )
df_demograficos['raca'].replace( 'Asiático', 2, inplace=True )
df_demograficos['raca'].replace( 'Branco', 3, inplace=True )
df_demograficos['raca'].replace( 'Outro', 4, inplace=True )

df_demograficos['ja_gravida'].replace( 'Não', 0, inplace=True )
df_demograficos['ja_gravida'].replace( 'Sim', 1, inplace=True )

df_demograficos['consumo_alcool'].replace('Bebe atualmente', 0, inplace=True)
df_demograficos['consumo_alcool'].replace('Bebia no passado', 1, inplace=True)
df_demograficos['consumo_alcool'].replace('não-informado', 2, inplace=True)
df_demograficos['consumo_alcool'].replace('Nunca bebeu', 3, inplace=True)

df_demograficos['historico_cancer'].replace('Não', 0, inplace=True)
df_demograficos['historico_cancer'].replace('Sim', 1, inplace=True)

df_demograficos['ultima_informacao_paciente'].replace( 'Vivo, SOE', 0, inplace=True )
df_demograficos['ultima_informacao_paciente'].replace( 'Vivo, com câncer', 1, inplace=True )
df_demograficos['ultima_informacao_paciente'].replace( 'Óbito por outras causas, SOE', 2, inplace=True )
df_demograficos['ultima_informacao_paciente'].replace( 'Obito por câncer', 3, inplace=True )

df_demograficos['grau_parentesco_primeiro'].replace('Unchecked', 0, inplace=True )
df_demograficos['grau_parentesco_primeiro'].replace('Checked', 1, inplace=True )

df_demograficos['grau_parentesco_segundo'].replace( 'Unchecked', 0, inplace=True )
df_demograficos['grau_parentesco_segundo'].replace( 'Checked', 1, inplace=True )

df_demograficos['grau_parentesco_terceiro'].replace( 'Unchecked', 0, inplace=True )
df_demograficos['grau_parentesco_terceiro'].replace( 'Checked', 1, inplace=True )

df_demograficos['tratamento'].replace( 'Terapia Adjuvante', 0, inplace=True )
df_demograficos['tratamento'].replace( 'Terapia Neoadjuvante', 1, inplace=True )

df_demograficos['tratamento'].replace( 'Unchecked', 0, inplace=True )
df_demograficos['tratamento'].replace( 'Checked', 1, inplace=True )

df_demograficos['anti_her2_neoadjuvante'].replace( 'Trastuzumabe', 0, inplace=True )
df_demograficos['anti_her2_neoadjuvante'].replace( 'Trastuzumabe + Pertuzumabe', 1, inplace=True )

df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Paliativo'].index)
df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Não fez quimioterapia'].index)

In [18]:
np.random.seed(73)



lista = [
    ["Sexo", 0, 1], 
    ["Escolaridade", 0, 5], ["idade_diagnostico"] , ["raca", 0, 4],
    ["ja_gravida", 0 , 1], ["consumo_alcool", 0 , 3],["gestacao_idade"],
    ["historico_cancer"],["grau_parentesco_primeiro", 0, 1],["grau_parentesco_segundo", 0, 1], 
    ["tempo_seguimento"], ["grau_parentesco_terceiro", 0, 1], ["anti_her2_neoadjuvante",0, 1]
         ]

lista_normal = []
l = 0

for i in range(0, len(lista)): 
  lista_normal.append(np.random.normal(df_demograficos[lista[i][0]].mean(skipna=True), 
                                       df_demograficos[lista[i][0]].std(skipna=True), 
                                       df_demograficos[f'{lista[i][0]}'].isnull().sum()))

  for j in range(0, len(lista_normal[i])):
    lista_normal[i][j] = round(lista_normal[i][j])

    if len(lista[i]) >= 2 : 
      if lista_normal[i][j] > lista[i][2]: 
        lista_normal[i][j] = lista[i][2]
      if lista_normal[i][j] < lista[i][1]: 
        lista_normal[i][j] = lista[i][1]


df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Paliativo'].index)
df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Não fez quimioterapia'].index)

tratamento_mean = df_demograficos['tratamento'].mean(skipna=True)
tratamento_std = df_demograficos['tratamento'].std(skipna=True)
null_tratamento = df_demograficos.tratamento.isnull().sum()
lista_normal_tratamento = np.random.normal(tratamento_mean, tratamento_std, null_tratamento)

for i in range(0, len(lista)):
  df_demograficos[f'{lista[i][0]}'] = df_demograficos[f'{lista[i][0]}'].fillna(pd.Series(lista_normal[i], index=df_demograficos[f'{lista[i][0]}'].index[df_demograficos[f'{lista[i][0]}'].isnull()]))

df_demograficos['tratamento'] = df_demograficos['tratamento'].fillna(pd.Series(lista_normal_tratamento, index=df_demograficos['tratamento'].index[df_demograficos['tratamento'].isnull()]))

df_demograficos

,Record ID,Escolaridade,idade_diagnostico,Sexo,raca,Data da última informação sobre o paciente,ultima_informacao_paciente,tempo_seguimento,ja_gravida,gestacao_idade,...,Atividade Física,consumo_alcool,historico_cancer,grau_parentesco_primeiro,grau_parentesco_segundo,grau_parentesco_terceiro,tratamento,Data da cirurgia,anti_her2_neoadjuvante,Data de início do tratamento quimioterapia
0,302,2.0,51.0,0.0,0.0,2014-04-26,3.0,2225.0,1.0,22.0,...,NaN,2.0,1.0,0,0,0,NaN,NaN,0.0,NaN
1,710,4.0,58.0,0.0,2.0,2016-11-17,0.0,3294.0,1.0,29.0,...,NaN,2.0,0.0,0,0,0,0,2009-09-04,0.0,2014-08-24
2,752,2.0,56.0,0.0,2.0,2019-05-02,0.0,4153.0,1.0,12.0,...,NaN,3.0,1.0,0,0,0,NaN,NaN,0.0,NaN
3,1367,2.0,63.0,0.0,1.0,2011-09-29,3.0,1331.0,1.0,26.0,...,NaN,3.0,0.0,0,0,0,NaN,2011-07-05,0.0,NaN
4,1589,3.0,42.0,0.0,1.0,2017-05-24,0.0,3290.0,1.0,26.0,...,NaN,1.0,2.0,0,0,0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123,0.0,57.0,0.0,2.0,2021-10-25,0.0,380.0,1.0,30.0,...,NaN,3.0,0.0,0,0,0,1,2021-07-04,0.0,2020-12-14
4268,82124,1.0,41.0,0.0,0.0,2021-01-21,3.0,138.0,1.0,27.0,...,NaN,2.0,0.0,0,0,0,NaN,NaN,0.0,NaN
4269,82131,4.0,59.0,0.0,3.0,2022-06-10,3.0,900.0,1.0,26.0,...,NaN,2.0,1.0,0,0,0,NaN,2020-12-23,0.0,NaN
4270,82205,1.0,29.0,0.0,2.0,2022-04-29,3.0,538.0,1.0,16.0,...,NaN,2.0,1.0,0,0,0,NaN,NaN,0.0,NaN


In [ ]:
df_histopatologia = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIHistopatologia_DATA_LABELS_2023-01-24_1924.csv")

df_histopatologia = df_histopatologia.drop(['Repeat Instrument', 
                                        'Repeat Instance',
                                        'Receptor de progesterona (quantificação %)',
                                        'Receptorde Estrogênio (quantificação %)',
                                        'Indice H (Receptorde progesterona)',
                                        'Ki67 (%)',
                                        ], axis=1)

df_histopatologia = df_histopatologia.rename(columns={"Record ID": "record_id"})

lista_RecorId = df_histopatologia['record_id'].unique()

df_novo_histopatologia = df_histopatologia 
df_novo = pd.DataFrame(columns= df_histopatologia.columns)

for i in lista_RecorId:
    df_query = df_histopatologia.query(f'record_id == {i}')
    idx_linha_max = df_query.apply(lambda x: x.count(), axis=1).idxmax()
    linha_max = df_query.loc[idx_linha_max]
    df_novo = df_novo.append(linha_max, ignore_index=True)

df_histopatologia = df_novo

df_histopatologia

,record_id,Diagnostico primario (tipo histológico),Grau histológico,Subtipo tumoral,Receptor de estrogênio,Receptor de progesterona,Ki67 (>14%),HER2 por IHC,HER2 por FISH
0,302,NaN,NaN,5.0,positivo,positivo,NaN,+++ (positivo),NaN
1,710,NaN,NaN,2.0,positivo,negativo,positivo,0 (negativo),NaN
2,752,NaN,NaN,3.0,NaN,NaN,NaN,+++ (positivo),NaN
3,1367,NaN,NaN,2.0,positivo,positivo,positivo,0 (negativo),NaN
4,1589,NÃO-ESPECIAL - Carcinoma de mama ductal invasi...,NaN,5.0,positivo,positivo,NaN,+++ (positivo),NaN
...,...,...,...,...,...,...,...,...,...
4267,82123,NaN,NaN,3.0,NaN,NaN,positivo,+++ (positivo),NaN
4268,82124,NaN,NaN,3.0,NaN,NaN,NaN,+++ (positivo),NaN
4269,82131,NaN,NaN,3.0,NaN,NaN,positivo,+++ (positivo),NaN
4270,82205,NaN,NaN,4.0,negativo,negativo,positivo,0 (negativo),NaN


In [ ]:
df_histopatologia = df_histopatologia.rename(columns={
                        "Grau histológico": "grau_histologico", 
                        "Subtipo tumoral": "subtipo_tumoral",
                        "Receptor de estrogênio": "receptor_estrogenio",
                        "Receptor de progesterona": "receptor_progesterona",
                        "Ki67 (>14%)": "ki67",
                        "HER2 por IHC": "her2_ihc",
                        "Diagnostico primario (tipo histológico)": "primeiro_diagnostico",
                        "HER2 por FISH": "her2_fish"
                        })

In [ ]:
df_histopatologia['primeiro_diagnostico'].replace('NÃO-ESPECIAL - Carcinoma de mama ductal invasivo (CDI)/SOE', 0, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace('Carcinoma de mama lobular invasivo', 1, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'outros', 2, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama metaplasico', 3, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama mucinoso', 4, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama papilifero', 5, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama medular', 6, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama micropapilar', 7, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama misto (ductal e lobular) invasivo', 8, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama cistico adenoide', 9, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma mamário invasivo multifocal', 10, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama misto (ductal e micropapilífero) invasivo', 11, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'CARCINOMA MAMÁRIO INVASIVO DO TIPO APÓCRINO', 12, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama lobular in situ', 13, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma lobular pleomórfico', 14, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Adenomioepitelioma maligno', 15, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Carcinoma de mama tubular', 16, inplace=True )
df_histopatologia['primeiro_diagnostico'].replace( 'Tumor PHYLLODES maligno', 17, inplace=True )

df_histopatologia['receptor_estrogenio'].replace('positivo', 0, inplace=True )
df_histopatologia['receptor_estrogenio'].replace('negativo', 1, inplace=True )
df_histopatologia['receptor_estrogenio'].replace('não realizado', 2, inplace=True )

df_histopatologia['receptor_progesterona'].replace('positivo', 0, inplace=True )
df_histopatologia['receptor_progesterona'].replace('negativo', 1, inplace=True )
df_histopatologia['receptor_progesterona'].replace('inconclusivo', 2, inplace=True )
df_histopatologia['receptor_progesterona'].replace('não realizado', 3, inplace=True )

df_histopatologia['ki67'].replace('positivo', 0, inplace=True )
df_histopatologia['ki67'].replace('negativo', 1, inplace=True )
df_histopatologia['ki67'].replace('inconclusivo', 2, inplace=True )
df_histopatologia['ki67'].replace('não realizado', 3, inplace=True )

df_histopatologia['her2_ihc'].replace('0 (negativo)', 0, inplace=True )
df_histopatologia['her2_ihc'].replace(' +++ (positivo)', 1, inplace=True )
df_histopatologia['her2_ihc'].replace(' ++ (duvidoso)', 2, inplace=True )
df_histopatologia['her2_ihc'].replace(' + (negativo)', 3, inplace=True )
df_histopatologia['her2_ihc'].replace('indeterminado', 4, inplace=True )

df_histopatologia['her2_fish'].replace('não realizado', 0, inplace=True )
df_histopatologia['her2_fish'].replace('amplificado', 1, inplace=True )
df_histopatologia['her2_fish'].replace('sem amplificação', 2, inplace=True )
df_histopatologia['her2_fish'].replace('duvidoso', 3, inplace=True )
df_histopatologia['her2_fish'].replace('reação não funcionou', 4, inplace=True )


In [ ]:
np.random.seed(73)

primeiro_diagnostico_mean = df_histopatologia["primeiro_diagnostico"].mean(skipna=True)
primeiro_diagnostico_std = df_histopatologia["primeiro_diagnostico"].std(skipna=True)
null_primeiro_diagnostico = df_histopatologia.primeiro_diagnostico.isnull().sum()
lista_normal_primeiro_diagnostico = np.random.normal(primeiro_diagnostico_mean, primeiro_diagnostico_std, null_primeiro_diagnostico)

grau_histologico_mean = df_histopatologia["grau_histologico"].mean(skipna=True)
grau_histologico_std = df_histopatologia["grau_histologico"].std(skipna=True)
null_grau_histologico = df_histopatologia.grau_histologico.isnull().sum()
lista_normal_grau_histologico = np.random.normal(grau_histologico_mean, grau_histologico_std, null_grau_histologico)

subtipo_tumoral_mean = df_histopatologia["subtipo_tumoral"].mean(skipna=True)
subtipo_tumoral_std = df_histopatologia["subtipo_tumoral"].std(skipna=True)
null_subtipo_tumoral = df_histopatologia.subtipo_tumoral.isnull().sum()
lista_normal_subtipo_tumoral = np.random.normal(subtipo_tumoral_mean, subtipo_tumoral_std, null_subtipo_tumoral)

receptor_estrogenio_mean = df_histopatologia["receptor_estrogenio"].mean(skipna=True)
receptor_estrogenio_std = df_histopatologia["receptor_estrogenio"].std(skipna=True)
null_receptor_estrogenio = df_histopatologia.receptor_estrogenio.isnull().sum()
lista_normal_receptor_estrogenio = np.random.normal(receptor_estrogenio_mean, receptor_estrogenio_std, null_receptor_estrogenio)

receptor_progesterona_mean = df_histopatologia["receptor_progesterona"].mean(skipna=True)
receptor_progesterona_std = df_histopatologia["receptor_progesterona"].std(skipna=True)
null_receptor_progesterona = df_histopatologia.receptor_progesterona.isnull().sum()
lista_normal_receptor_progesterona = np.random.normal(receptor_progesterona_mean, receptor_progesterona_std, null_receptor_progesterona)

ki67_mean = df_histopatologia["ki67"].mean(skipna=True)
ki67_std = df_histopatologia["ki67"].std(skipna=True)
null_ki67 = df_histopatologia.ki67.isnull().sum()
lista_normal_ki67 = np.random.normal(ki67_mean, ki67_std, null_ki67)

her2_ihc_mean = df_histopatologia["her2_ihc"].mean(skipna=True)
her2_ihc_std = df_histopatologia["her2_ihc"].std(skipna=True)
null_her2_ihc = df_histopatologia.her2_ihc.isnull().sum()
lista_normal_her2_ihc = np.random.normal(her2_ihc_mean, her2_ihc_std, null_her2_ihc)

her2_fish_mean = df_histopatologia["her2_fish"].mean(skipna=True)
her2_fish_std = df_histopatologia["her2_fish"].std(skipna=True)
null_her2_fish = df_histopatologia.her2_fish.isnull().sum()
lista_normal_her2_fish = np.random.normal(her2_fish_mean, her2_fish_std, null_her2_fish)

for i in range(0, null_primeiro_diagnostico):
  lista_normal_primeiro_diagnostico[i] = round(lista_normal_primeiro_diagnostico[i])
  if lista_normal_primeiro_diagnostico[i] > 17: 
    lista_normal_primeiro_diagnostico[i] = 17
  if lista_normal_primeiro_diagnostico[i] < 0: 
    lista_normal_primeiro_diagnostico[i] = 0

for i in range(0, null_grau_histologico):
  lista_normal_grau_histologico[i] = round(lista_normal_grau_histologico[i])

for i in range(0, null_subtipo_tumoral):
  lista_normal_subtipo_tumoral[i] = round(lista_normal_subtipo_tumoral[i])
  if lista_normal_subtipo_tumoral[i] > 5: 
    lista_normal_subtipo_tumoral[i] = 5

for i in range(0, null_receptor_estrogenio):
  lista_normal_receptor_estrogenio[i] = round(lista_normal_receptor_estrogenio[i])
  if lista_normal_receptor_estrogenio[i] > 2: 
    lista_normal_receptor_estrogenio[i] = 2
  if lista_normal_receptor_estrogenio[i] < 0: 
    lista_normal_receptor_estrogenio[i] = 0

for i in range(0, null_receptor_progesterona):
  lista_normal_receptor_progesterona[i] = round(lista_normal_receptor_progesterona[i])
  if lista_normal_receptor_progesterona[i] > 3: 
    lista_normal_receptor_progesterona[i] = 3
  if lista_normal_receptor_progesterona[i] < 0: 
    lista_normal_receptor_progesterona[i] = 0

for i in range(0, null_ki67):
  lista_normal_ki67[i] = round(lista_normal_ki67[i])
  if lista_normal_ki67[i] > 3: 
    lista_normal_ki67[i] = 3
  if lista_normal_ki67[i] < 0: 
    lista_normal_ki67[i] = 0

for i in range(0, null_her2_ihc):
  lista_normal_her2_ihc[i] = round(lista_normal_her2_ihc[i])
  if lista_normal_her2_ihc[i] > 4: 
    lista_normal_her2_ihc[i] = 4
  if lista_normal_her2_ihc[i] < 0: 
    lista_normal_her2_ihc[i] = 0

for i in range(0, null_her2_fish):
  lista_normal_her2_fish[i] = round(lista_normal_her2_fish[i])
  if lista_normal_her2_fish[i] > 4: 
    lista_normal_her2_fish[i] = 4
  if lista_normal_her2_fish[i] < 0: 
    lista_normal_her2_fish[i] = 0

df_histopatologia["primeiro_diagnostico"] = df_histopatologia["primeiro_diagnostico"].fillna(pd.Series(lista_normal_primeiro_diagnostico, index=df_histopatologia["primeiro_diagnostico"].index[df_histopatologia["primeiro_diagnostico"].isnull()]))
df_histopatologia["grau_histologico"] = df_histopatologia["grau_histologico"].fillna(pd.Series(lista_normal_grau_histologico, index=df_histopatologia["grau_histologico"].index[df_histopatologia["grau_histologico"].isnull()]))
df_histopatologia["subtipo_tumoral"] = df_histopatologia["subtipo_tumoral"].fillna(pd.Series(lista_normal_subtipo_tumoral, index=df_histopatologia["subtipo_tumoral"].index[df_histopatologia["subtipo_tumoral"].isnull()]))
df_histopatologia["receptor_estrogenio"] = df_histopatologia["receptor_estrogenio"].fillna(pd.Series(lista_normal_receptor_estrogenio, index=df_histopatologia["receptor_estrogenio"].index[df_histopatologia["receptor_estrogenio"].isnull()]))
df_histopatologia["receptor_progesterona"] = df_histopatologia["receptor_progesterona"].fillna(pd.Series(lista_normal_receptor_progesterona, index=df_histopatologia["receptor_progesterona"].index[df_histopatologia["receptor_progesterona"].isnull()]))
df_histopatologia["ki67"] = df_histopatologia["ki67"].fillna(pd.Series(lista_normal_ki67, index=df_histopatologia["ki67"].index[df_histopatologia["ki67"].isnull()]))
df_histopatologia["her2_ihc"] = df_histopatologia["her2_ihc"].fillna(pd.Series(lista_normal_her2_ihc, index=df_histopatologia["her2_ihc"].index[df_histopatologia["her2_ihc"].isnull()]))
df_histopatologia["her2_fish"] = df_histopatologia["her2_fish"].fillna(pd.Series(lista_normal_her2_fish, index=df_histopatologia["her2_fish"].index[df_histopatologia["her2_fish"].isnull()]))

df_histopatologia

,record_id,primeiro_diagnostico,grau_histologico,subtipo_tumoral,receptor_estrogenio,receptor_progesterona,ki67,her2_ihc,her2_fish
0,302,1.0,1.0,5.0,0.0,0.0,0.0,1.0,1.0
1,710,4.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0
2,752,5.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0
3,1367,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
4,1589,0.0,2.0,5.0,0.0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...
4267,82123,0.0,2.0,3.0,1.0,1.0,0.0,1.0,0.0
4268,82124,1.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0
4269,82131,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0
4270,82205,0.0,2.0,4.0,1.0,1.0,0.0,0.0,1.0


In [ ]:
df_registro_tumo = df_registro_tumo.drop(['Repeat Instrument', 
                                        'Repeat Instance'
                                        ], axis=1)

In [ ]:
df_registro_tumo = df_registro_tumo.rename(columns={
                        "Record ID" : "record_id",
                        "Código da Topografia (CID-O)": "cid_o", 
                        "Código da Morfologia de acordo com o CID-O": "morfologia_cid_o",
                        "Estadio Clínico  ": "estadio_clinico",
                        "Grupo de Estadio Clínico": "grupo_estadio_clinico",
                        "Metastase ao DIAGNOSTICO - CID-O #4 ": "metastase_cido4",
                        "Classificação TNM Clínico - M": "classificacao_tnm_m",
                        "Classificação TNM Clínico - N": "classificacao_tnm_n",
                        "Classificação TNM Patológico - N": "classificacao_tnm_n_patologico",
                        "Classificação TNM Patológico - T": "classificacao_tnm_t_patologico",
                        "Com recidiva à distância": "recidiva_distancia",
                        "Com recidiva regional": "recidiva_regional",
                        "Com recidiva local": "recidiva_local",
                        "Combinação dos Tratamentos Realizados no Hospital": "combinacao_tratamentos",
                        "Data de Recidiva": "data_recidiva",
                        "Lateralidade do tumor": "lateralidade_tumor",
                        "Data do tratamento": "data_tratamento",
                        "Tempo desde o diagnóstico até a primeira recidiva  ": "tempo_diagnostico",
                        "Descrição da Morfologia de acordo com o CID-O  (CID-O - 3ª edição)": "descricao_morfologia",
                        "Descrição da Topografia ": "descricao_topografia"
                        })

df_registro_tumo["grupo_estadio_clinico"].replace("0", 0, inplace=True )
df_registro_tumo["grupo_estadio_clinico"].replace("I", 1, inplace=True )
df_registro_tumo["grupo_estadio_clinico"].replace("II", 2, inplace=True )
df_registro_tumo["grupo_estadio_clinico"].replace("III", 3, inplace=True )
df_registro_tumo["grupo_estadio_clinico"].replace("IV", 4, inplace=True )
df_registro_tumo["grupo_estadio_clinico"].replace("Y: Na", 5, inplace=True )
df_registro_tumo["grupo_estadio_clinico"].replace("X - nao foi possivel determinar", 6, inplace=True)

df_registro_tumo["lateralidade_tumor"].replace("Esquerda", 0, inplace=True )
df_registro_tumo["lateralidade_tumor"].replace("Direita", 1, inplace=True )
df_registro_tumo["lateralidade_tumor"].replace("não se aplica", 2, inplace=True )
df_registro_tumo["lateralidade_tumor"].replace("Bilateral", 3, inplace=True )

df_registro_tumo["combinacao_tratamentos"].replace("Cirurgia + Radio + Quimio + Hormonio", 0, inplace=True )
df_registro_tumo["combinacao_tratamentos"].replace("Cirurgia + Quimioterapia", 1, inplace=True )
df_registro_tumo["combinacao_tratamentos"].replace("Outras combinações", 2, inplace=True )
df_registro_tumo["combinacao_tratamentos"].replace("Cirurgia + Radio + Quimio", 3, inplace=True )
df_registro_tumo["combinacao_tratamentos"].replace("Cirurgia + Radioterapia", 4, inplace=True )
df_registro_tumo["combinacao_tratamentos"].replace("Quimioterapia", 5, inplace=True )
df_registro_tumo["combinacao_tratamentos"].replace("Radioterapia + Quimioterapia", 6, inplace=True)
df_registro_tumo["combinacao_tratamentos"].replace("Nenhum tratamento", 7, inplace=True)
df_registro_tumo["combinacao_tratamentos"].replace("Radioterapia", 8, inplace=True)
df_registro_tumo["combinacao_tratamentos"].replace("Cirurgia", 9, inplace=True) 

df_registro_tumo["classificacao_tnm_m"].replace("0", 0, inplace=True )
df_registro_tumo["classificacao_tnm_m"].replace("1", 1, inplace=True )
df_registro_tumo["classificacao_tnm_m"].replace("Y: Na", 2, inplace=True )
df_registro_tumo["classificacao_tnm_m"].replace("X - nao foi possivel determinar", 3, inplace=True )
df_registro_tumo["classificacao_tnm_m"].replace("1B", 4, inplace=True )
df_registro_tumo["classificacao_tnm_m"].replace("1A", 5, inplace=True )

df_registro_tumo["classificacao_tnm_n"].replace("0", 0, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("1", 1, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("2", 2, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("3", 3, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("1A", 4, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("2A", 5, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("3A", 6, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("1B", 7, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("2B", 8, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("3B", 9, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("3C", 10, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("X - nao foi possivel determinar", 11, inplace=True )
df_registro_tumo["classificacao_tnm_n"].replace("Y: Na", 12, inplace=True )

df_registro_tumo["classificacao_tnm_t_patologico"].replace("2", 0, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("1C", 1, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("3", 2, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("1", 3, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("1B", 4, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("4B", 5, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("IV", 6, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("IS", 7, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("4D", 8, inplace=True ) 
df_registro_tumo["classificacao_tnm_t_patologico"].replace("4C", 9, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("1MIC", 10, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("Y: Na", 11, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("2C", 12, inplace=True )
df_registro_tumo["classificacao_tnm_t_patologico"].replace("1A", 13, inplace=True )

df_registro_tumo["classificacao_tnm_n_patologico"].replace("0", 0, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("1", 1, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("2", 2, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("3", 2, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("2A", 3, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("3A", 4, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("3B", 5, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("3C", 6, inplace=True )
df_registro_tumo["classificacao_tnm_n_patologico"].replace("Y: Na", 7, inplace=True)
df_registro_tumo["classificacao_tnm_n_patologico"].replace("X - nao foi possivel determinar", 8, inplace=True)

df_registro_tumo["recidiva_distancia"].replace("Não", 0, inplace=True )
df_registro_tumo["recidiva_distancia"].replace("Sim", 1, inplace=True )

df_registro_tumo["recidiva_regional"].replace("Não", 0, inplace=True )
df_registro_tumo["recidiva_regional"].replace("Sim", 1, inplace=True )

df_registro_tumo["recidiva_local"].replace("Não", 0, inplace=True )
df_registro_tumo["recidiva_local"].replace("Sim", 1, inplace=True )

In [ ]:
lista_RecorId = df_registro_tumo['record_id'].unique()

df_novo_histopatologia = df_registro_tumo 
df_novo = pd.DataFrame(columns= df_registro_tumo.columns)

for i in lista_RecorId:
    df_query = df_registro_tumo.query(f'record_id == {i}')
    idx_linha_max = df_query.apply(lambda x: x.count(), axis=1).idxmax()
    linha_max = df_query.loc[idx_linha_max]
    df_novo = df_novo.append(linha_max, ignore_index=True)

df_registro_tumo = df_novo

df_registro_tumo

,record_id,Data da primeira consulta institucional [dt_pci],Data do diagnóstico,cid_o,morfologia_cid_o,estadio_clinico,grupo_estadio_clinico,Classificação TNM Clínico - T,classificacao_tnm_n,classificacao_tnm_m,...,Local de Recidiva a distancia/ metastase #2 - CID-O - Topografia,Local de Recidiva a distancia/ metastase #3 - CID-O - Topografia,Local de Recidiva a distancia/ metastase #4 - CID-O - Topografia,descricao_morfologia,descricao_topografia,classificacao_tnm_n_patologico,classificacao_tnm_t_patologico,recidiva_distancia,recidiva_regional,recidiva_local
0,302,2008-03-22,2008-03-23,C504,85003.0,IIA,2.0,2,0.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,NaN,NaN,0.0,0.0,0.0
1,710,2006-11-11,2007-11-11,C508,85003.0,IIIA,3.0,3,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA LESAO SOBREPOSTA DA,NaN,NaN,0.0,1.0,1.0
2,752,2007-09-25,2007-12-18,C509,84803.0,IIA,2.0,2,0.0,0.0,...,NaN,NaN,NaN,ADENOCARCINOMA MUCINOSO,MAMA SOE (EXCLUI PELE DA MAMA C44.5),8.0,0.0,0.0,0.0,0.0
3,1367,2008-02-03,2008-02-06,C505,85003.0,IIA,2.0,1,1.0,0.0,...,C50 - Mama,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE INFERIOR EXTERNO DA,1.0,13.0,0.0,1.0,0.0
4,1589,2008-05-15,2008-05-21,C508,85003.0,IIB,2.0,2,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA LESAO SOBREPOSTA DA,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123,2020-12-04,2020-10-10,C504,85003.0,IIB,NaN,3,0.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,NaN,NaN,0.0,0.0,0.0
4268,82124,2020-06-20,2020-09-05,C509,85203.0,IV,NaN,4D,1.0,1.0,...,NaN,NaN,NaN,CARCINOMA LOBULAR SOE,MAMA SOE (EXCLUI PELE DA MAMA C44.5),NaN,NaN,0.0,0.0,0.0
4269,82131,2020-11-01,2019-12-23,C502,85203.0,IIIA,NaN,3,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA LOBULAR SOE,MAMA QUADRANTE SUPERIOR INTERNO DA,NaN,NaN,0.0,0.0,0.0
4270,82205,2021-02-28,2020-11-07,C504,85003.0,IV,NaN,4D,1.0,1.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,NaN,NaN,0.0,0.0,1.0


In [ ]:
np.random.seed(73)

null_cid_o = df_registro_tumo.cid_o.isnull().sum()
lista_normal_cid_o = []

null_descricao_topografia = df_registro_tumo.descricao_topografia.isnull().sum()
lista_normal_descricao_topografia = []

null_descricao_morfologia = df_registro_tumo.descricao_morfologia.isnull().sum()
lista_normal_descricao_morfologia = []

grupo_estadio_clinico_mean = df_registro_tumo["grupo_estadio_clinico"].mean(skipna=True)
grupo_estadio_clinico_std = df_registro_tumo["grupo_estadio_clinico"].std(skipna=True)
null_grupo_estadio_clinico = df_registro_tumo.grupo_estadio_clinico.isnull().sum()
lista_normal_grupo_estadio_clinico = np.random.normal(grupo_estadio_clinico_mean, grupo_estadio_clinico_std, null_grupo_estadio_clinico)

combinacao_tratamentos_mean = df_registro_tumo["combinacao_tratamentos"].mean(skipna=True)
combinacao_tratamentos_std = df_registro_tumo["combinacao_tratamentos"].std(skipna=True)
null_combinacao_tratamentos = df_registro_tumo.combinacao_tratamentos.isnull().sum()
lista_normal_combinacao_tratamentos = np.random.normal(combinacao_tratamentos_mean, combinacao_tratamentos_std, null_combinacao_tratamentos)

tempo_diagnostico_mean = df_registro_tumo["tempo_diagnostico"].mean(skipna=True)
tempo_diagnostico_std = df_registro_tumo["tempo_diagnostico"].std(skipna=True)
null_tempo_diagnostico = df_registro_tumo.tempo_diagnostico.isnull().sum()
lista_normal_tempo_diagnostico = np.random.normal(tempo_diagnostico_mean, tempo_diagnostico_std, null_tempo_diagnostico)
 
lateralidade_tumor_mean = df_registro_tumo["lateralidade_tumor"].mean(skipna=True)
lateralidade_tumor_std = df_registro_tumo["lateralidade_tumor"].std(skipna=True)
null_lateralidade_tumor = df_registro_tumo.lateralidade_tumor.isnull().sum()
lista_normal_lateralidade_tumor = np.random.normal(lateralidade_tumor_mean, lateralidade_tumor_std, null_lateralidade_tumor)

classificacao_tnm_m_mean = df_registro_tumo["classificacao_tnm_m"].mean(skipna=True)
classificacao_tnm_m_std = df_registro_tumo["classificacao_tnm_m"].std(skipna=True)
null_classificacao_tnm_m = df_registro_tumo.classificacao_tnm_m.isnull().sum()
lista_normal_classificacao_tnm_m = np.random.normal(classificacao_tnm_m_mean, classificacao_tnm_m_std, null_classificacao_tnm_m)

classificacao_tnm_n_mean = df_registro_tumo["classificacao_tnm_n"].mean(skipna=True)
classificacao_tnm_n_std = df_registro_tumo["classificacao_tnm_n"].std(skipna=True)
null_classificacao_tnm_n = df_registro_tumo.classificacao_tnm_n.isnull().sum()
lista_normal_classificacao_tnm_n = np.random.normal(classificacao_tnm_n_mean, classificacao_tnm_n_std, null_classificacao_tnm_n)

morfologia_cid_o_mean = df_registro_tumo["morfologia_cid_o"].mean(skipna=True)
morfologia_cid_o_std = df_registro_tumo["morfologia_cid_o"].std(skipna=True)
null_morfologia_cid_o = df_registro_tumo.morfologia_cid_o.isnull().sum()
lista_normal_morfologia_cid_o = np.random.normal(morfologia_cid_o_mean, morfologia_cid_o_std, null_morfologia_cid_o)

classificacao_tnm_n_patologico_mean = df_registro_tumo["classificacao_tnm_n_patologico"].mean(skipna=True)
classificacao_tnm_n_patologico_std = df_registro_tumo["classificacao_tnm_n_patologico"].std(skipna=True)
null_classificacao_tnm_n_patologico = df_registro_tumo.classificacao_tnm_n_patologico.isnull().sum()
lista_normal_classificacao_tnm_n_patologico = np.random.normal(classificacao_tnm_n_patologico_mean, classificacao_tnm_n_patologico_std, null_classificacao_tnm_n_patologico)

classificacao_tnm_t_patologico_mean = df_registro_tumo["classificacao_tnm_t_patologico"].mean(skipna=True)
classificacao_tnm_t_patologico_std = df_registro_tumo["classificacao_tnm_t_patologico"].std(skipna=True)
null_classificacao_tnm_t_patologico = df_registro_tumo.classificacao_tnm_t_patologico.isnull().sum()
lista_normal_classificacao_tnm_t_patologico = np.random.normal(classificacao_tnm_t_patologico_mean, classificacao_tnm_t_patologico_std, null_classificacao_tnm_t_patologico)

grupo_estadio_clinico_mean = df_registro_tumo["grupo_estadio_clinico"].mean(skipna=True)
grupo_estadio_clinico_std = df_registro_tumo["grupo_estadio_clinico"].std(skipna=True)
null_grupo_estadio_clinico = df_registro_tumo.grupo_estadio_clinico.isnull().sum()
lista_normal_grupo_estadio_clinico = np.random.normal(grupo_estadio_clinico_mean, grupo_estadio_clinico_std, null_grupo_estadio_clinico)

recidiva_distancia_mean = df_registro_tumo["recidiva_distancia"].mean(skipna=True)
recidiva_distancia_std = df_registro_tumo["recidiva_distancia"].std(skipna=True)
null_recidiva_distancia = df_registro_tumo.recidiva_distancia.isnull().sum()
lista_normal_recidiva_distancia = np.random.normal(recidiva_distancia_mean, recidiva_distancia_std, null_recidiva_distancia)

recidiva_regional_mean = df_registro_tumo["recidiva_regional"].mean(skipna=True)
recidiva_regional_std = df_registro_tumo["recidiva_regional"].std(skipna=True)
null_recidiva_regional = df_registro_tumo.recidiva_regional.isnull().sum()
lista_normal_recidiva_regional = np.random.normal(recidiva_regional_mean, recidiva_regional_std, null_recidiva_regional)

recidiva_local_mean = df_registro_tumo["recidiva_local"].mean(skipna=True)
recidiva_local_std = df_registro_tumo["recidiva_local"].std(skipna=True)
null_recidiva_local = df_registro_tumo.recidiva_local.isnull().sum()
lista_normal_recidiva_local = np.random.normal(recidiva_local_mean, recidiva_local_std, null_recidiva_local)

for i in range(0, null_cid_o):
  lista_normal_cid_o.append("C509")

for i in range(0, null_descricao_morfologia): 
  lista_normal_descricao_morfologia.append("CARCINOMA DUCTAL INFILTRANTE SOE")

for i in range(0, null_descricao_topografia): 
  lista_normal_descricao_topografia.append("MAMA SOE (EXCLUI PELE DA MAMA C44.5)")

for i in range(0, null_grupo_estadio_clinico):
  lista_normal_grupo_estadio_clinico[i] = round(lista_normal_grupo_estadio_clinico[i])
  if lista_normal_grupo_estadio_clinico[i] > 6: 
    lista_normal_grupo_estadio_clinico[i] = 6
  if lista_normal_grupo_estadio_clinico[i] < 0: 
    lista_normal_grupo_estadio_clinico[i] = 0

for i in range(0, null_lateralidade_tumor):
  lista_normal_lateralidade_tumor[i] = round(lista_normal_lateralidade_tumor[i])
  if lista_normal_lateralidade_tumor[i] > 22: 
    lista_normal_lateralidade_tumor[i] = 22
  if lista_normal_lateralidade_tumor[i] < 0: 
    lista_normal_lateralidade_tumor[i] = 0

for i in range(0, null_tempo_diagnostico):
  lista_normal_tempo_diagnostico[i] = round(lista_normal_tempo_diagnostico[i])
  if lista_normal_tempo_diagnostico[i] > 22: 
    lista_normal_tempo_diagnostico[i] = 22
  if lista_normal_tempo_diagnostico[i] < 0: 
    lista_normal_tempo_diagnostico[i] = 0
  
for i in range(0, null_combinacao_tratamentos):
  lista_normal_combinacao_tratamentos[i] = round(lista_normal_combinacao_tratamentos[i])
  if lista_normal_combinacao_tratamentos[i] > 22: 
    lista_normal_combinacao_tratamentos[i] = 22
  if lista_normal_combinacao_tratamentos[i] < 0: 
    lista_normal_combinacao_tratamentos[i] = 0

for i in range(0, null_classificacao_tnm_t_patologico):
  lista_normal_classificacao_tnm_t_patologico[i] = round(lista_normal_classificacao_tnm_t_patologico[i])
  if lista_normal_classificacao_tnm_t_patologico[i] > 22: 
    lista_normal_classificacao_tnm_t_patologico[i] = 22
  if lista_normal_classificacao_tnm_t_patologico[i] < 0: 
    lista_normal_classificacao_tnm_t_patologico[i] = 0

for i in range(0, null_classificacao_tnm_n_patologico):
  lista_normal_classificacao_tnm_n_patologico[i] = round(lista_normal_classificacao_tnm_n_patologico[i])
  if lista_normal_classificacao_tnm_n_patologico[i] > 22: 
    lista_normal_classificacao_tnm_n_patologico[i] = 22
  if lista_normal_classificacao_tnm_n_patologico[i] < 0: 
    lista_normal_classificacao_tnm_n_patologico[i] = 0

for i in range(0, null_grupo_estadio_clinico):
  lista_normal_grupo_estadio_clinico[i] = round(lista_normal_grupo_estadio_clinico[i])
  if lista_normal_grupo_estadio_clinico[i] > 22: 
    lista_normal_grupo_estadio_clinico[i] = 22
  if lista_normal_grupo_estadio_clinico[i] < 0: 
    lista_normal_grupo_estadio_clinico[i] = 0

for i in range(0, null_classificacao_tnm_m):
  lista_normal_classificacao_tnm_m[i] = round(lista_normal_classificacao_tnm_m[i])
  if lista_normal_classificacao_tnm_m[i] > 5: 
    lista_normal_classificacao_tnm_m[i] = 5
  if lista_normal_classificacao_tnm_m[i] < 0: 
    lista_normal_classificacao_tnm_m[i] = 0

for i in range(0, null_classificacao_tnm_n):
  lista_normal_classificacao_tnm_n[i] = round(lista_normal_classificacao_tnm_n[i])
  if lista_normal_classificacao_tnm_n[i] > 9: 
    lista_normal_classificacao_tnm_n[i] = 9
  if lista_normal_classificacao_tnm_n[i] < 0: 
    lista_normal_classificacao_tnm_n[i] = 0

for i in range(0, null_morfologia_cid_o):
  lista_normal_morfologia_cid_o[i] = round(lista_normal_morfologia_cid_o[i])
  if lista_normal_morfologia_cid_o[i] > 22: 
    lista_normal_morfologia_cid_o[i] = 22
  if lista_normal_morfologia_cid_o[i] < 0: 
    lista_normal_morfologia_cid_o[i] = 0

for i in range(0, null_recidiva_distancia):
  lista_normal_recidiva_distancia[i] = round(lista_normal_recidiva_distancia[i])
  if lista_normal_recidiva_distancia[i] > 1: 
    lista_normal_recidiva_distancia[i] = 1
  if lista_normal_recidiva_distancia[i] < 0: 
    lista_normal_recidiva_distancia[i] = 0

for i in range(0, null_recidiva_regional):
  lista_normal_recidiva_regional[i] = round(lista_normal_recidiva_regional[i])
  if lista_normal_recidiva_regional[i] > 1: 
    lista_normal_recidiva_regional[i] = 1
  if lista_normal_recidiva_regional[i] < 0: 
    lista_normal_recidiva_regional[i] = 0

for i in range(0, null_recidiva_local):
  lista_normal_recidiva_local[i] = round(lista_normal_recidiva_local[i])
  if lista_normal_recidiva_local[i] > 1: 
    lista_normal_recidiva_local[i] = 1
  if lista_normal_recidiva_local[i] < 0: 
    lista_normal_recidiva_local[i] = 0

df_registro_tumo["cid_o"] = df_registro_tumo["cid_o"].fillna(pd.Series(lista_normal_cid_o, index=df_registro_tumo["cid_o"].index[df_registro_tumo["cid_o"].isnull()]))
df_registro_tumo["grupo_estadio_clinico"] = df_registro_tumo["grupo_estadio_clinico"].fillna(pd.Series(lista_normal_grupo_estadio_clinico, index=df_registro_tumo["grupo_estadio_clinico"].index[df_registro_tumo["grupo_estadio_clinico"].isnull()]))
df_registro_tumo["classificacao_tnm_m"] = df_registro_tumo["classificacao_tnm_m"].fillna(pd.Series(lista_normal_classificacao_tnm_m, index=df_registro_tumo["classificacao_tnm_m"].index[df_registro_tumo["classificacao_tnm_m"].isnull()]))
df_registro_tumo["classificacao_tnm_n"] = df_registro_tumo["classificacao_tnm_n"].fillna(pd.Series(lista_normal_classificacao_tnm_n, index=df_registro_tumo["classificacao_tnm_n"].index[df_registro_tumo["classificacao_tnm_n"].isnull()]))
df_registro_tumo["recidiva_distancia"] = df_registro_tumo["recidiva_distancia"].fillna(pd.Series(lista_normal_recidiva_distancia, index=df_registro_tumo["recidiva_distancia"].index[df_registro_tumo["recidiva_distancia"].isnull()]))
df_registro_tumo["recidiva_regional"] = df_registro_tumo["recidiva_regional"].fillna(pd.Series(lista_normal_recidiva_regional, index=df_registro_tumo["recidiva_regional"].index[df_registro_tumo["recidiva_regional"].isnull()]))
df_registro_tumo["recidiva_local"] = df_registro_tumo["recidiva_local"].fillna(pd.Series(lista_normal_recidiva_local, index=df_registro_tumo["recidiva_local"].index[df_registro_tumo["recidiva_local"].isnull()]))
df_registro_tumo["morfologia_cid_o"] = df_registro_tumo["morfologia_cid_o"].fillna(pd.Series(lista_normal_morfologia_cid_o, index=df_registro_tumo["morfologia_cid_o"].index[df_registro_tumo["morfologia_cid_o"].isnull()]))
df_registro_tumo["combinacao_tratamentos"] = df_registro_tumo["combinacao_tratamentos"].fillna(pd.Series(lista_normal_combinacao_tratamentos, index=df_registro_tumo["combinacao_tratamentos"].index[df_registro_tumo["combinacao_tratamentos"].isnull()]))
df_registro_tumo["lateralidade_tumor"] = df_registro_tumo["lateralidade_tumor"].fillna(pd.Series(lista_normal_lateralidade_tumor, index=df_registro_tumo["lateralidade_tumor"].index[df_registro_tumo["lateralidade_tumor"].isnull()]))
df_registro_tumo["tempo_diagnostico"] = df_registro_tumo["tempo_diagnostico"].fillna(pd.Series(lista_normal_tempo_diagnostico, index=df_registro_tumo["tempo_diagnostico"].index[df_registro_tumo["tempo_diagnostico"].isnull()]))
df_registro_tumo["descricao_morfologia"] = df_registro_tumo["descricao_morfologia"].fillna(pd.Series(lista_normal_descricao_morfologia, index=df_registro_tumo["descricao_morfologia"].index[df_registro_tumo["descricao_morfologia"].isnull()]))
df_registro_tumo["descricao_topografia"] = df_registro_tumo["descricao_topografia"].fillna(pd.Series(lista_normal_descricao_topografia, index=df_registro_tumo["descricao_topografia"].index[df_registro_tumo["descricao_topografia"].isnull()]))
df_registro_tumo["classificacao_tnm_n_patologico"] = df_registro_tumo["classificacao_tnm_n_patologico"].fillna(pd.Series(lista_normal_classificacao_tnm_n_patologico, index=df_registro_tumo["classificacao_tnm_n_patologico"].index[df_registro_tumo["classificacao_tnm_n_patologico"].isnull()]))
df_registro_tumo["classificacao_tnm_t_patologico"] = df_registro_tumo["classificacao_tnm_t_patologico"].fillna(pd.Series(lista_normal_classificacao_tnm_t_patologico, index=df_registro_tumo["classificacao_tnm_t_patologico"].index[df_registro_tumo["classificacao_tnm_t_patologico"].isnull()]))

df_registro_tumo

<ipython-input-92-4cee6de13bd6>:177: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_registro_tumo["cid_o"] = df_registro_tumo["cid_o"].fillna(pd.Series(lista_normal_cid_o, index=df_registro_tumo["cid_o"].index[df_registro_tumo["cid_o"].isnull()]))
<ipython-input-92-4cee6de13bd6>:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_registro_tumo["descricao_morfologia"] = df_registro_tumo["descricao_morfologia"].fillna(pd.Series(lista_normal_descricao_morfologia, index=df_registro_tumo["descricao_morfologia"].index[df_registro_tumo["descricao_morfologia"].isnull()]))
<ipython-input-92-4cee6de13bd6>:189: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to 

,record_id,Data da primeira consulta institucional [dt_pci],Data do diagnóstico,cid_o,morfologia_cid_o,estadio_clinico,grupo_estadio_clinico,Classificação TNM Clínico - T,classificacao_tnm_n,classificacao_tnm_m,...,Local de Recidiva a distancia/ metastase #2 - CID-O - Topografia,Local de Recidiva a distancia/ metastase #3 - CID-O - Topografia,Local de Recidiva a distancia/ metastase #4 - CID-O - Topografia,descricao_morfologia,descricao_topografia,classificacao_tnm_n_patologico,classificacao_tnm_t_patologico,recidiva_distancia,recidiva_regional,recidiva_local
0,302,2008-03-22,2008-03-23,C504,85003.0,IIA,2.0,2,0.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,0.0,0.0,0.0,0.0,0.0
1,710,2006-11-11,2007-11-11,C508,85003.0,IIIA,3.0,3,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA LESAO SOBREPOSTA DA,1.0,7.0,0.0,1.0,1.0
2,752,2007-09-25,2007-12-18,C509,84803.0,IIA,2.0,2,0.0,0.0,...,NaN,NaN,NaN,ADENOCARCINOMA MUCINOSO,MAMA SOE (EXCLUI PELE DA MAMA C44.5),8.0,0.0,0.0,0.0,0.0
3,1367,2008-02-03,2008-02-06,C505,85003.0,IIA,2.0,1,1.0,0.0,...,C50 - Mama,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE INFERIOR EXTERNO DA,1.0,13.0,0.0,1.0,0.0
4,1589,2008-05-15,2008-05-21,C508,85003.0,IIB,2.0,2,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA LESAO SOBREPOSTA DA,1.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123,2020-12-04,2020-10-10,C504,85003.0,IIB,4.0,3,0.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,1.0,2.0,0.0,0.0,0.0
4268,82124,2020-06-20,2020-09-05,C509,85203.0,IV,3.0,4D,1.0,1.0,...,NaN,NaN,NaN,CARCINOMA LOBULAR SOE,MAMA SOE (EXCLUI PELE DA MAMA C44.5),1.0,4.0,0.0,0.0,0.0
4269,82131,2020-11-01,2019-12-23,C502,85203.0,IIIA,1.0,3,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA LOBULAR SOE,MAMA QUADRANTE SUPERIOR INTERNO DA,0.0,6.0,0.0,0.0,0.0
4270,82205,2021-02-28,2020-11-07,C504,85003.0,IV,4.0,4D,1.0,1.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,3.0,0.0,0.0,0.0,1.0
